In [1]:
import pandas as pd
import numpy as np
import pickle as p

In [2]:
data_all = pd.read_csv('data_all.csv')

In [3]:
np.random.seed(41)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

In [4]:
item_subset

[31, 2259, 1203, 5073, 102, 5501, 3818, 4270, 3767, 197]

### `/train`

In [5]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [6]:
# Model Training Result
r.json()['result']

'Success'

In [7]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict(json.loads(r.json()['cv_acc']))
cv_accuracies

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,31.0,45.218984,0.646753,13.531055,0.299234,19.612755
1,102.0,176.608918,0.657642,41.242826,0.233526,58.734683
2,197.0,55.278581,-0.004060,21.837249,0.395040,28.438235
3,1203.0,198.355324,0.772615,31.884889,0.160746,45.929615
4,2259.0,97.155863,0.582326,26.962036,0.277513,33.459526
5,3767.0,286.172376,0.745849,47.444712,0.165791,63.297534
6,3818.0,78.448462,0.659540,16.395637,0.208999,22.030331
7,4270.0,45.090722,-0.111343,16.486173,0.365622,20.390819
8,5073.0,115.369817,0.505503,29.863306,0.258849,38.809853
9,5501.0,464.741615,0.650251,94.465141,0.203264,125.479912


In [12]:
import sys

In [13]:
sys.getsizeof(data)

24151299

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [10]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'31': 2.0,
 '102': 9.45,
 '197': 10.4,
 '1203': 3.2,
 '2259': 12.95,
 '3767': 10.45,
 '3818': 10.3,
 '4270': 8.3,
 '5073': 3.65,
 '5501': 3.75}

In [11]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_102,122.0,91.0
Qty_1203,227.0,222.0
Qty_197,79.0,74.0
Qty_2259,94.0,63.0
Qty_31,67.0,78.0
Qty_3767,269.0,267.0
Qty_3818,69.0,82.0
Qty_4270,37.0,67.0
Qty_5073,189.0,232.0
Qty_5501,558.0,453.0
